In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import timm
import torch
from unittest.mock import patch

from src.model.swin_transformer_v2_pseudo_3d import SwinTransformerV2Pseudo3d, map_pretrained_2d_to_pseudo_3d

In [3]:
model_2d = timm.create_model(
    'swinv2_tiny_window8_256.ms_in1k', 
    pretrained=True,
).cuda()
x = torch.randn(1, 3, 256, 256).cuda()
y = model_2d(x)

/root/miniconda3/envs/scrolls/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([1, 64, 64, 96])


In [4]:
with patch('timm.models.swin_transformer_v2.SwinTransformerV2', SwinTransformerV2Pseudo3d):
    model_pseudo_3d = timm.create_model(
        'swinv2_tiny_window8_256.ms_in1k', 
        pretrained=False,
        window_size=(8, 8, 16),
        img_size=(256, 256, 64),
    ).cuda()
x = torch.randn(1, 3, 256, 256, 64).cuda()
y = model_pseudo_3d(x)

In [5]:
model_2d_state_dict = model_2d.state_dict()
model_pseudo_3d_state_dict = model_pseudo_3d.state_dict()
for key, value in model_2d_state_dict.items():
    if key in model_pseudo_3d_state_dict:
        if value.shape == model_pseudo_3d_state_dict[key].shape:
            print(f'{key}: {value.shape} -> OK')
        else:
            print(f'{key}: {value.shape} -> {model_pseudo_3d_state_dict[key].shape}')
    else:
        print(f'{key}: {value.shape} -> NOT FOUND')

patch_embed.proj.weight: torch.Size([96, 3, 4, 4]) -> torch.Size([96, 3, 4, 4, 4])
patch_embed.proj.bias: torch.Size([96]) -> OK
patch_embed.norm.weight: torch.Size([96]) -> OK
patch_embed.norm.bias: torch.Size([96]) -> OK
layers.0.blocks.0.attn.logit_scale: torch.Size([3, 1, 1]) -> OK
layers.0.blocks.0.attn.q_bias: torch.Size([96]) -> OK
layers.0.blocks.0.attn.v_bias: torch.Size([96]) -> OK
layers.0.blocks.0.attn.cpb_mlp.0.weight: torch.Size([512, 2]) -> torch.Size([512, 3])
layers.0.blocks.0.attn.cpb_mlp.0.bias: torch.Size([512]) -> OK
layers.0.blocks.0.attn.cpb_mlp.2.weight: torch.Size([3, 512]) -> OK
layers.0.blocks.0.attn.qkv.weight: torch.Size([288, 96]) -> OK
layers.0.blocks.0.attn.proj.weight: torch.Size([96, 96]) -> OK
layers.0.blocks.0.attn.proj.bias: torch.Size([96]) -> OK
layers.0.blocks.0.norm1.weight: torch.Size([96]) -> OK
layers.0.blocks.0.norm1.bias: torch.Size([96]) -> OK
layers.0.blocks.0.mlp.fc1.weight: torch.Size([384, 96]) -> OK
layers.0.blocks.0.mlp.fc1.bias: tor

No-matches are `patch_embed.proj` (Conv2d -> Conv3d) and `layers.0.blocks.0.attn.cpb_mlp.0` (relative position bias mapping MLP for Z dim) layers' weights and biases, algthough biases shapes match. 

- Conv layer's weight: `torch.Size([96, 3, 4, 4]) -> torch.Size([96, 3, 4, 4, 4])`

- MLP's weight: `torch.Size([512, 2]) -> torch.Size([512, 3])`

For conv layer proposal is to repeat weights along 3rd dimension and scale them down by patch size along Z dim (4) and keep bias term intact. E. g. if the image is just repeated along Z dim, then the 3D patch embedding in such case will be equal to 2D patch embedding of non-repeated patch.

For relative position bias proposal is to calculate weights for new dimention as mean of weights of previous two and keep the bias intact. No invariancy for that case.

**Note**: it needs additional investigation whether low-rank of the obtained weights is a problem.

In [6]:
model = map_pretrained_2d_to_pseudo_3d(model_2d, model_pseudo_3d)

patch_embed.proj.weight: torch.Size([96, 3, 4, 4]) -> torch.Size([96, 3, 4, 4, 4])
layers.0.blocks.0.attn.cpb_mlp.0.weight: torch.Size([512, 2]) -> torch.Size([512, 3])
